# Train_data txt2vec
- 학습 데이터 내 text들을 vector로 미리 변환
- 변환 후 json 형식으로 저장
- 학습시에는 저장한 vector를 불러와서 사용할 예정 (하나의 노트북에서 모든 작업을 수행할 경우 RAM을 너무 많이 씀)

In [ ]:
import json 

with open('data/brand_dict.json', 'r') as f:
    brand_dict = json.load( f)

with open('data/cat_3_dict.json', 'r') as f:
    cat_3_dict = json.load(f)

In [ ]:
train_df = pd.read_csv('data/train.csv')

In [ ]:
train_df = train_df[['content_id', 'content_name', 'content_keyword', 'content_cat_1', 'content_cat_2', 'content_cat_3']].drop_duplicates()

### 제목 : 정규표현식으로 한글만 남기기

In [ ]:
import re
def brand_(name, brand_dict):
    for key in brand_dict.keys():
        for value in brand_dict[key]:
            if value in name.split(' '):
                name = name.replace(value, key)
    return name

def re_kor_brand(s):
    s = brand_(s, brand_dict)
    pre = re.compile('[^ 가-힣]')
    result = pre.sub(' ', s) 
    while '  ' in result:
        result = result.replace('  ', ' ')
    result = result.split(' ')
    result = [r for r in result if r!='']
    return result

In [ ]:
name_list = train_df.content_name.apply(re_kor_brand)
name_list

0                                [당일발송, 아이폰, 대량판매, 등급별초특가]
2                                [당일발송, 갤럭시, 대량판매, 등급별초특가]
3                                [당일발송, 갤럭시, 대량판매, 등급별초특가]
6                             [한정판매, 아이폰, 블랙, 급, 중고폰, 공기계]
8                                  [한정판매, 아이폰, 미니, 급, 화이트]
                                ...                       
735683                            [빈티지푸, 정품, 셔츠, 맛집, 인정이요]
735685    [정품, 원터치텐트, 낚시텐트, 자외선차단텐트, 방수텐트, 캠핑텐트, 간편, 자동텐트]
735688     [남자운동화, 남자스니커즈, 키높이운동화, 캐주얼운동화, 남자신발, 워킹화, 런닝화]
735691                           [버그만, 스즈끼, 년형, 오토바이, 판매중]
735693                                    [올뉴카니발, 인승, 팝니다]
Name: content_name, Length: 1404, dtype: object

### 제목 단어모음 만들기

In [ ]:
def get_voca_cnt(voca_list):
    total_vocabs = []
    for t in voca_list:
        total_vocabs.extend(t)
    total_unique, total_count = np.unique(total_vocabs, return_counts=True)
    vocab_cnt = pd.DataFrame({'name': total_unique, 'cnt': total_count})
    return vocab_cnt.sort_values('cnt', ascending=False)

In [ ]:
name_cnt = get_voca_cnt(name_list)
name_cnt

,name,cnt
188,급,408
1093,아이폰,214
57,갤럭시,141
800,블랙,113
316,당일발송,113
...,...,...
799,블라이크리스탈,1
797,브이라이트,1
796,브이넥,1
795,브이,1


### 키워드 : "," 기준으로 구분해주기

In [ ]:
keyword_list = train_df.content_keyword.fillna('').apply(lambda x: x.split(','))
keyword_list

0                 [아이폰SE2중고, 아이폰se2공기계, SE2중고, SE2, 아이폰SE2]
2                   [중고갤럭시s20, s20중고폰, s20공기계, 갤럭시s20, s20]
3              [갤럭시S10E중고, S10E중고폰, s10e공기계, s10e, 갤럭시s10e]
6                                       [아이폰SE2, 아이폰SE2그레이]
8                                                        []
                                ...                        
735683                           [폴로셔츠, 단색셔츠, 폴로랄프로렌, 옥스포드]
735685    [원터치텐트간편텐트, 낚시텐트등산텐트, 자외선차단방수텐트, 자동텐트감성캠핑, 캠핑용...
735688    [남자남성신발구두, 워킹화농구화슬립온, 구두로퍼슬리퍼샌들, 스니커즈운동화신발, 칸투...
735691                    [익사이팅400, 버그만, 티맥스, c600, 포르자300]
735693                 [올뉴카니발, 카니발, 카니발중고, 올뉴카니발중고, 카니발중고차]
Name: content_keyword, Length: 1404, dtype: object

### 제목 단어모음 만들기

In [ ]:
keyword_cnt = get_voca_cnt(keyword_list)
keyword_cnt

,name,cnt
0,,86
907,그래픽카드,53
1950,애슐리상점,51
582,vga,50
1162,도금,40
...,...,...
1140,대구,1
1139,당일지급,1
1138,당일배송,1
1137,단체점퍼,1


## 제목-키워드 단어 모음 만들기

In [ ]:
total_vocab = pd.merge(name_cnt, keyword_cnt,how='outer', on='name').fillna(0)
total_vocab['cnt'] = (total_vocab['cnt_x'] + total_vocab['cnt_y']).astype(int)
total_vocab = total_vocab[['name', 'cnt']]
total_vocab

,name,cnt
0,급,408
1,아이폰,222
2,갤럭시,160
3,블랙,113
4,당일발송,113
...,...,...
4275,대형청소기,1
4276,대납개통,1
4277,대나무체인목걸이,1
4278,대구경북중고휠,1


### 카테고리

In [ ]:
from ast import literal_eval

In [ ]:
A = [cat for cat in cat_names]

In [ ]:
AA = literal_eval(",".join(A).replace('[', '').replace(']', ''))

In [ ]:
total_dict = {}
for dic in AA:
    total_dict = {**total_dict, **dic}

In [ ]:
cat_dict = dict(zip([int(k) for k in total_dict.keys()], total_dict.values()))

In [ ]:
with open('data/cat_num2name.json', 'w') as f:
    json.dump(cat_dict, f)

In [ ]:
cat_list1 = train_df.content_cat_1.apply(lambda x: [cat.replace('기타', 'etc').replace(' ', '') for cat in x.split('/')])
cat_list2 = train_df.content_cat_2.apply(lambda x: cat_dict[x]).apply(lambda x: x.replace('/', ' ')).apply(lambda x: cat_3_dict[x] if x in cat_3_dict.keys() else x).apply(lambda x: x.split(' '))
cat_list3 = train_df.content_cat_3.apply(lambda x: cat_dict[x]).apply(lambda x: x.replace('/', ' ')).apply(lambda x: cat_3_dict[x] if x in cat_3_dict.keys() else x).apply(lambda x: x.split(' '))
cat_list = cat_list1 + cat_list2 + cat_list3

## Description : Kkma, re 📑

In [ ]:
kkma = Kkma()

In [ ]:
# kkma로 명사 추출 후 단어 모음에 있는 단어만 남기기
def kkma_voca(sentence):
    voca_list = kkma.nouns(sentence)
    return [voca for voca in voca_list if voca in total_vocab.name.tolist()]

# 정규표현식으로 전처리 후 단어 모음에 있는 단어만 남기기
def re_voca(sentence):
    voca_list = re_kor(sentence)
    return [voca for voca in voca_list if voca in total_vocab.name.tolist()]

# kkma, 정규표현식으로 나온 각 리스트 합쳐서 한번에
def re_kkma_voca(sentence):
    voca_list1 = kkma.nouns(sentence)
    voca_list2 = re_kor_brand(sentence)
    voca_list = voca_list1 + voca_list2
    return [voca for voca in voca_list if voca in total_vocab.name.tolist()]

- 상세설명 결측 채우기

In [ ]:
train_description = pd.read_csv('data/train_description.csv')

In [ ]:
t_des = train_description.drop_duplicates()

In [ ]:
train_df

,content_id,content_name,content_keyword,content_cat_1,content_cat_2,content_cat_3
0,137078818,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,"아이폰SE2중고,아이폰se2공기계,SE2중고,SE2,아이폰SE2",디지털/가전,600700,600700001
2,137118198,#당일발송#갤럭시S20 128G 대량판매 등급별초특가!,"중고갤럭시s20,s20중고폰,s20공기계,갤럭시s20,s20",디지털/가전,600700,600700001
3,137118119,#당일발송#갤럭시 S10e 128G 대량판매 등급별초특가!,"갤럭시S10E중고,S10E중고폰,s10e공기계,s10e,갤럭시s10e",디지털/가전,600700,600700001
6,161996984,#한정판매#아이폰SE 64GB 블랙 SS급 중고폰 공기계[88941],"아이폰SE2,아이폰SE2그레이",디지털/가전,600700,600700001
8,163059060,#한정판매#아이폰12미니 64G SS급 화이트 [10012],NaN,디지털/가전,600700,600700001
...,...,...,...,...,...,...
735683,77262424,<빈티지푸> 정품 polo 셔츠 맛집 인정이요!,"폴로셔츠,단색셔츠,폴로랄프로렌,옥스포드",남성의류,320040,320040100
735685,98827336,정품 원터치텐트 낚시텐트 자외선차단텐트 방수텐트 캠핑텐트 간편 자동텐트,"원터치텐트간편텐트,낚시텐트등산텐트,자외선차단방수텐트,자동텐트감성캠핑,캠핑용품그늘막",스포츠/레저,700200,700200004
735688,135475928,@남자운동화 남자스니커즈 키높이운동화 캐주얼운동화 남자신발 워킹화 런닝화,"남자남성신발구두,워킹화농구화슬립온,구두로퍼슬리퍼샌들,스니커즈운동화신발,칸투칸캐주얼화헬스",신발,405100,405100
735691,162434266,"버그만650 스즈끼 07년형 오토바이 판매중 ""BMMOTO""","익사이팅400,버그만,티맥스,c600,포르자300",차량/오토바이,750800,750800200


In [ ]:
t_des= t_des.fillna('')

In [ ]:
text_df.description = text_df.description.fillna('!')

In [ ]:
t_des

,content_id,content_text
0,137078818,
2,137118198,상품정보\n👍🏻 중고노트북 전문 판매상점 테크몰 👍🏻\n☑️믿음과 신용 💯 으로 ...
3,137118119,
6,161996984,상품정보\n더바비Love' 밴드로 초대합니다.\nhttps://band.us/n/a...
8,163059060,"상품정보\n★ 번개장터 구매고객 특별이벤트 ★\n\n1,구입후 중고폰 100일 제품..."
...,...,...
735683,77262424,
735685,98827336,상품정보\n찐찐 정품입니다 \n골든구스 슈퍼스타 \n밑창 보강했던 신발입니다 \n브...
735688,135475928,상품정보\n*액정 잔상 기스 없고 외관 깨끗한 상태(테두리구석 아주미세한 기스)\n...
735691,162434266,상품정보\n판매상품은 미개봉 새상품입니다.\n번개페이로 구매하시면 11시이전 거래 ...


- 전처리 진행

In [ ]:
# kkma 전처리
# desc_list = text_df.description.apply(kkma_voca)
# # kkma + re 전처리
desc_list = t_des.content_text.apply(re_kkma_voca)

# Word2Vec 학습

### 각 컨텐츠마다 한 문장 만들기(이름 + 키워드 + 상세설명 + 카테고리)

In [ ]:
total_sentences = name_list + keyword_list + cat_list #+ desc_list
total_sentences = total_sentences.apply(lambda x: [xx for xx in x if xx != ''])

In [ ]:
total_sentences

0         [당일발송, 아이폰, 대량판매, 등급별초특가, 아이폰SE2중고, 아이폰se2공기계,...
2         [당일발송, 갤럭시, 대량판매, 등급별초특가, 중고갤럭시s20, s20중고폰, s2...
3         [당일발송, 갤럭시, 대량판매, 등급별초특가, 갤럭시S10E중고, S10E중고폰, ...
6         [한정판매, 아이폰, 블랙, 급, 중고폰, 공기계, 아이폰SE2, 아이폰SE2그레이...
8               [한정판매, 아이폰, 미니, 급, 화이트, 디지털, 가전, 모바일, 스마트폰]
                                ...                        
735683    [빈티지푸, 정품, 셔츠, 맛집, 인정이요, 폴로셔츠, 단색셔츠, 폴로랄프로렌, 옥...
735685    [정품, 원터치텐트, 낚시텐트, 자외선차단텐트, 방수텐트, 캠핑텐트, 간편, 자동텐...
735688    [남자운동화, 남자스니커즈, 키높이운동화, 캐주얼운동화, 남자신발, 워킹화, 런닝화...
735691    [버그만, 스즈끼, 년형, 오토바이, 판매중, 익사이팅400, 버그만, 티맥스, c...
735693    [올뉴카니발, 인승, 팝니다, 올뉴카니발, 카니발, 카니발중고, 올뉴카니발중고, 카...
Length: 1404, dtype: object

## Text2vec
- 사전에 학습시킨 FastText 모델을 통해 text를 벡터로

In [ ]:
from gensim.models import FastText
model_ft = FastText.load('model/ft.model')

In [ ]:
def text2vec(text):
    total_vec = np.zeros(100,)
    cnt = 0
    for t in text:
        try:
            total_vec += model_ft.wv.word_vec(t)
            cnt += 1
        except:
            pass
    total_vec /= cnt

    return total_vec

In [ ]:
train_df['vector_no'] = total_sentences.apply(text2vec)

In [ ]:
id2tvec = train_df[['content_id', 'vector_no']]

- DeepFM input 형식을 고려하여 list로 저장

In [ ]:
# id2tvec['vector'].tolist()
train_text2vec = dict(zip(id2tvec['content_id'].tolist(), [list(vec) for vec in id2tvec['vector_no'].tolist()]))

- 결과 json 형식으로 저장

In [ ]:
with open('data/train_text2vec_no.json', 'w') as f:
    json.dump(train_text2vec, f)